# 01 — Data Ingestion & Validation

Fetches stock prices (yfinance), macro indicators (FRED), and FOMC event data.
Builds and validates the master dataframe. Exports `prices_daily.csv` for Tableau.

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.data_fetcher import (
    fetch_stock_prices,
    fetch_macro_data,
    load_fomc_events,
    get_hike_events,
    build_master_dataframe,
    export_prices_for_tableau,
    MAG7_TICKERS,
)

sns.set_theme(style='whitegrid')
%matplotlib inline

## 1. Fetch Stock Prices

In [ ]:
prices = fetch_stock_prices()
print(f"Shape: {prices.shape}")
print(f"Date range: {prices.index.min()} to {prices.index.max()}")
print(f"Tickers: {list(prices.columns)}")
print(f"\nNull counts:\n{prices.isnull().sum()}")
prices.tail()

## 2. Fetch Macro Data (FRED)

In [ ]:
# Requires FRED_API_KEY in .env file
# Register free at: https://fred.stlouisfed.org/docs/api/api_key.html
macro = fetch_macro_data()
print(f"Shape: {macro.shape}")
print(f"Date range: {macro.index.min()} to {macro.index.max()}")
print(f"\nColumns: {list(macro.columns)}")
print(f"\nNull counts:\n{macro.isnull().sum()}")
macro.tail()

## 3. Load FOMC Events

In [ ]:
fomc = load_fomc_events()
hikes = get_hike_events(fomc)

print(f"Total rate changes: {len(fomc)}")
print(f"  Hikes: {len(fomc[fomc['direction'] == 'hike'])}")
print(f"  Cuts:  {len(fomc[fomc['direction'] == 'cut'])}")
print(f"\nHike cycles:")
print(f"  2015-2018: {len(hikes[(hikes['date'] >= '2015-01-01') & (hikes['date'] <= '2018-12-31')])} hikes")
print(f"  2022-2023: {len(hikes[(hikes['date'] >= '2022-01-01') & (hikes['date'] <= '2023-12-31')])} hikes")
fomc

## 4. Build Master Dataframe

In [ ]:
master = build_master_dataframe(prices, macro, fomc)
print(f"Shape: {master.shape}")
print(f"Columns: {list(master.columns)}")
print(f"\nRate regime distribution:")
print(master['rate_regime'].value_counts())
print(f"\nFOMC event days (T+1): {master['fomc_event'].sum()}")
master.tail()

## 5. Visualizations

In [ ]:
# Mag 7 price history (normalized to 100 at start)
fig, ax = plt.subplots(figsize=(14, 7))

for ticker in MAG7_TICKERS:
    normalized = (prices[ticker] / prices[ticker].dropna().iloc[0]) * 100
    ax.plot(normalized, label=ticker, linewidth=1.2)

# Add vertical lines for rate hikes
for _, row in hikes.iterrows():
    ax.axvline(row['date'], color='red', alpha=0.15, linewidth=0.8)

ax.set_title('Magnificent 7 — Normalized Price History (2015–Present)', fontsize=14)
ax.set_ylabel('Price (indexed to 100)')
ax.legend(loc='upper left')
ax.set_xlabel('')
plt.tight_layout()
plt.show()

In [ ]:
# Fed Funds Rate timeline with rate hike markers
fig, ax1 = plt.subplots(figsize=(14, 5))

ax1.fill_between(master.index, master['fed_rate'], alpha=0.3, color='steelblue', label='Fed Funds Rate')
ax1.plot(master.index, master['fed_rate'], color='steelblue', linewidth=1.5)

for _, row in hikes.iterrows():
    ax1.axvline(row['date'], color='red', alpha=0.4, linewidth=1, linestyle='--')

ax1.set_title('Federal Funds Rate with Rate Hike Events', fontsize=14)
ax1.set_ylabel('Rate (%)')
ax1.legend(loc='upper left')
plt.tight_layout()
plt.show()

## 6. Validation Checks

In [ ]:
# Validation
print("=== Validation Checks ===")

# No duplicate dates
assert not prices.index.duplicated().any(), "Duplicate dates in prices!"
print("[PASS] No duplicate dates in prices")

# All Mag 7 tickers present
for t in MAG7_TICKERS:
    assert t in prices.columns, f"Missing ticker: {t}"
print(f"[PASS] All {len(MAG7_TICKERS)} Mag 7 tickers present")

# Date range starts from 2015
assert prices.index.min().year == 2015, "Prices don't start from 2015"
print(f"[PASS] Price data starts from {prices.index.min().date()}")

# Macro data has key columns
for col in ['fed_rate', 'cpi', 'treasury_10y']:
    assert col in macro.columns, f"Missing macro column: {col}"
print(f"[PASS] All macro columns present: {list(macro.columns)}")

# FOMC hikes count is reasonable
assert len(hikes) >= 15, f"Only {len(hikes)} hike events — expected ~20"
print(f"[PASS] {len(hikes)} hike events found")

print("\n=== All validation checks passed ===")

## 7. Export for Tableau

In [ ]:
long_df = export_prices_for_tableau(master)
long_df.head(10)